In [1]:
from mxnet.gluon import nn
from mxnet import nd


def get_net():
    net = nn.HybridSequential()
    with net.name_scope():
        net.add(nn.Dense(256, activation='relu'),
                nn.Dense(128, activation='relu'),
                nn.Dense(2))
    net.initialize()
    return net


x = nd.random_normal(shape=(1, 512))
net = get_net()
net(x)


[[0.08811311 0.06387279]]
<NDArray 1x2 @cpu(0)>

In [2]:
net.hybridize()
net(x)


[[0.08811311 0.06387279]]
<NDArray 1x2 @cpu(0)>

In [3]:
# Hybrid即符号式执行。符号式执行的python代码性能更高。
# 在mxnet当中，只有继承自HybridBlock的计算层才会被优化，
# 其他的会被跳过

from time import time


def bench(net, x):
    start = time()
    for i in range(1000):
        y = net(x)
    nd.waitall()
    return time() - start


net_ = get_net()
print('Before hybridizing,: %f sec' %(bench(net_, x)))
net_.hybridize()
print('After hybridizing: %f sec' % (bench(net_, x)))

Before hybridizing,: 0.312200 sec
After hybridizing: 0.218381 sec


In [4]:
from mxnet import sym

x = sym.var('data')
y = net(x)
print(y.tojson())

{
  "nodes": [
    {
      "op": "null", 
      "name": "data", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "hybridsequential0_dense0_weight", 
      "attrs": {
        "__dtype__": "0", 
        "__lr_mult__": "1.0", 
        "__shape__": "(256, 0)", 
        "__storage_type__": "0", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "hybridsequential0_dense0_bias", 
      "attrs": {
        "__dtype__": "0", 
        "__init__": "zeros", 
        "__lr_mult__": "1.0", 
        "__shape__": "(256,)", 
        "__storage_type__": "0", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "FullyConnected", 
      "name": "hybridsequential0_dense0_fwd", 
      "attrs": {
        "flatten": "True", 
        "no_bias": "False", 
        "num_hidden": "256"
      }, 
      "inputs": [[0, 0, 0], [1, 0, 0], [2, 0, 0]]
    }, 
    {
      "op": "Activation", 
      "name": "hybridse

In [5]:
class HybirdNet(nn.HybridBlock):

    def __init__(self, *args, **kwargs):
        super(HybirdNet, self).__init__(*args, **kwargs)
        with self.name_scope():
            self.fc1 = nn.Dense(10)
            self.fc2 = nn.Dense(2)

    def hybrid_forward(self, F, x):
        print(F)
        print(x)
        x = F.relu(self.fc1(x))
        print(x)
        return self.fc2(x)

In [6]:
net = HybirdNet()
net.initialize()
x = nd.random_normal(shape=(1, 4))
y = net(x)

<module 'mxnet.ndarray' from 'D:\\workspaces\\py3\\lib\\site-packages\\mxnet\\ndarray\\__init__.py'>

[[ 0.02184284 -0.31464806 -0.33364916 -0.6471778 ]]
<NDArray 1x4 @cpu(0)>

[[0.         0.02384557 0.         0.01206701 0.         0.02765122
  0.         0.03072213 0.02471942 0.        ]]
<NDArray 1x10 @cpu(0)>


In [7]:
net.hybridize()
y = net(x)

<module 'mxnet.symbol' from 'D:\\workspaces\\py3\\lib\\site-packages\\mxnet\\symbol\\__init__.py'>
<Symbol data>
<Symbol hybirdnet0_relu0>
